# Computational Exercise II

## Imports

In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

## Set Up

### Problem Instance

In [2]:
def create_problem_instance(m, n):
    # Creates a positive semidefinite matrix Q with integer coeffients in [-1, 1]
    Q = np.random.randint(-1, 2, size=(m + n, m + n))
    Q = np.matmul(Q, Q.transpose())

    # Creates a positive semidefinite matrix A with integer coeffients in [-1, 1]
    A = np.random.randint(-1, 2, size=(m + n, m + n))
    A = np.matmul(A, A.transpose())

    # Creates vector of dimensions m + n x 1 with coefficients
    # from a uniform distribution in the interval (0, 1)
    c1 = np.random.uniform(0, 1, (m + n))

    # Creates the remaining constants of the problem
    sy = n / 2
    sx = m
    b1 = 10 * (m + n)

    problem_data = {
        'Q': Q,
        'A': A,
        'c1': c1,
        'sy': sy,
        'sx': sx,
        'b1': b1
    }
    return problem_data

### Utility Functions

In [3]:
def random_continuous_point(m):
    return np.random.rand(m)

In [4]:
def random_binary_point(n):
    return np.random.randint(0, 2, size=n)

In [5]:
def random_point(m, n):
    x = random_continuous_point(m)
    y = random_binary_point(n)
    z = np.concatenate((x, y))
    return z

In [6]:
def random_viable_point(m, n, problem_data):
    # Extracts problem data
    Q = problem_data["Q"]
    A = problem_data["A"]
    c1 = problem_data["c1"]
    sy = problem_data["sy"]
    sx = problem_data["sx"]
    b1 = problem_data["b1"]
    
    # Creates a new model
    model = gp.Model("minlp")

    # Creates variables
    x = model.addVars(m, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="x")
    y = model.addVars(n, lb=0, ub=1, vtype=GRB.BINARY, name="y")
    z = [x[i] for i in range(m)] + [y[i] for i in range(n)]
    z_arr = np.array(z)

    # Defines objective function
    objective = 0

    # Sets objective
    model.setObjective(objective, GRB.MINIMIZE)

    # Adds constraint z^TAx - c^Tz <= b1
    model.addConstr(
        np.dot(z_arr.T, np.dot(A, z_arr)) - np.dot(c1.T, z_arr) <= b1,
        name="constraint_ineq1"
    )

    # Adds constraint sx/5 <= 1^Tx
    model.addConstr(
        (1/5)*sx
        - gp.quicksum(x[i] for i in range(m)) <= 0,
        name="constraint_ineq2"
    )

    # Adds constraint 1^Tx <= sx
    model.addConstr(
        gp.quicksum(x[i] for i in range(m)) 
        - sx <= 0,
        name="constraint_ineq3"
    )

    # Adds constraint x >= 0
    for i in range(m):
        model.addConstr(
            x[i] >= 0,
            name=f"constraint_ineq4_{i+1}"
        )

    # Adds constraint of contraint 1^Ty = sy
    model.addConstr(gp.quicksum(y[i] for i in range(n)) - sy == 0, name=f"constraint_eq")

    # Configures log to not show messages on terminal
    model.setParam("OutputFlag", 0)

    # Defines SoftMemLimit (in GigaBytes)
    model.setParam('SoftMemLimit', 8)

    # Optimizes model
    model.optimize()

    # Checks if model was optimized successfully
    if model.status == GRB.OPTIMAL:
        # Extracts model variables
        x_values = np.array([x[i].X for i in range(m)])
        y_values = np.array([y[i].X for i in range(n)])
        z_values = [x_values[i] for i in range(m)] + [y_values[i] for i in range(n)]
        return z_values
    else:
        raise RuntimeError(f"Model was not optimized successfully. Status: {model.status}")

In [7]:
def is_xy_viable(x, y, m, n, problem_data):
    # Extracts problem data
    Q = problem_data["Q"]
    A = problem_data["A"]
    c1 = problem_data["c1"]
    sy = problem_data["sy"]
    sx = problem_data["sx"]
    b1 = problem_data["b1"]

    # Sets variable z
    z = [x[i] for i in range(m)] + [y[i] for i in range(n)]
    z_arr = np.array(z)

    # Evaluates constraints
    constr1 = np.dot(z_arr.T, np.dot(A, z_arr)) - np.dot(c1.T, z_arr) <= b1
    constr2 = sum(y) == sy
    constr3 = (1/5)*sx <= sum(x)
    constr4 = sum(x) <= sx
    constr5 = all([x[i] >= 0 for i in range(m)])
    constrs = [constr1, constr2, constr3, constr4, constr5]
    
    return all(constrs)

In [8]:
def fxy(x, y, Q):
    z = [x[i] for i in range(m)] + [y[i] for i in range(n)]
    z_arr = np.array(z)
    f_eval = np.dot(z_arr.T, np.dot(Q, z_arr))
    return f_eval

In [9]:
def fz(z, Q):
    z_arr = np.array(z)
    f_eval = np.dot(z_arr.T, np.dot(Q, z_arr))
    return f_eval

In [10]:
import psutil

def insufficient_memory():
    # Gets total memory and available memory
    memory = psutil.virtual_memory()

    # Calculates percentage of available memory
    percentage_available_memory = memory.available / memory.total * 100

    if (percentage_available_memory < 0.05):
        return True
    return False

In [11]:
def infeasible_constraints(x, y, m, n, problem_data, L):
    # Sets alpha
    alpha = fxy(x, y, problem_data["Q"])

    # Extracts problem data
    Q = problem_data["Q"]
    A = problem_data["A"]
    c1 = problem_data["c1"]
    sy = problem_data["sy"]
    sx = problem_data["sx"]
    b1 = problem_data["b1"]

    # Sets variable z
    z = [x[i] for i in range(m)] + [y[i] for i in range(n)]
    z_arr = np.array(z)

    # Creates list of violated constraints
    VL = []

    # Evaluates constraints
    for k in range(len(L)):
        zk = np.array(L[k])

        constr0 = 2 * np.dot(zk.T, np.dot(Q, z_arr - zk)) + np.dot(zk.T, np.dot(Q, zk)) <= alpha
        if not constr0:
            VL.append(f"constraint_obj_{k+1}")
        
        constr1 = 2 * np.dot(zk.T, np.dot(A, z_arr - zk)) - np.dot(c1.T, z_arr - zk) + np.dot(zk.T, np.dot(A, zk)) - np.dot(c1.T, zk) - b1 <= 0
        if not constr1:
            VL.append(f"constraint_ineq1_{k+1}")

        xk = zk[:m]

    constr2 = (1/5)*sx - sum(x[i] for i in range(m)) <= 0
    if not constr2:
        VL.append(f"constraint_ineq2")

    constr3 = sum(x[i] for i in range(m)) - sx <= 0
    if not constr3:
        VL.append(f"constraint_ineq3")
        
    for i in range(m):
        constr4 = x[i] >= 0,
        if not constr4:
            VL.append(f"constraint_ineq4_{i+1}")
    
    constr5 = sum(y[i] for i in range(n)) - sy == 0
    if not constr5:
        VL.append(f"constraint_eq")

    return VL

In [12]:
def numpy_to_latex(array):
    # Checks if array is a vector or matric
    if array.ndim == 1:  # Vector
        latex_code = '\\begin{bmatrix}\n'
        latex_code += ' \\\\ \n'.join(map(str, array))  # Adds vector elements
        latex_code += '\n\\end{bmatrix}'
    elif array.ndim == 2:  # Matrix
        latex_code = '\\begin{bmatrix}\n'
        for row in array:
            latex_code += ' & '.join(map(str, row)) + ' \\\\ \n'  # Adds each matrix row
        latex_code += '\\end{bmatrix}'
    else:
        raise ValueError("Input must be a vector or a matrix.")
    
    return latex_code

### Algorithms

In [13]:
def master_problem(m, n, problem_data, L, B=[]):
    # Extracts problem data
    Q = problem_data["Q"]
    A = problem_data["A"]
    c1 = problem_data["c1"]
    sy = problem_data["sy"]
    sx = problem_data["sx"]
    b1 = problem_data["b1"]

    # Creates a new model
    model = gp.Model("milp")

    # Creates variables
    alpha = model.addVar(lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="alpha")
    x = model.addVars(m, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="x")
    y = model.addVars(n, lb=0, ub=1, vtype=GRB.BINARY, name="y")
    z = [x[i] for i in range(m)] + [y[i] for i in range(n)]
    z_arr = np.array(z)

    # Defines objective function
    objective = alpha

    # Sets objective
    model.setObjective(objective, GRB.MINIMIZE)

    for k in range(len(L)):
        zk = np.array(L[k])
        # Adds constraints of linearized objetive function from problem P
        model.addConstr(
            np.dot(zk.T, np.dot(Q, z_arr - zk)) + np.dot(zk.T, np.dot(Q, zk)) <= alpha,
            name=f"constraint_obj_{k+1}"
        )
        # Adds constraint of linearized constraint z^TAz - c^Tz <= b1 from problem P
        model.addConstr(
            2 * np.dot(zk.T, np.dot(A, z_arr - zk)) - np.dot(c1.T, z_arr - zk) + np.dot(zk, np.dot(A, zk)) - np.dot(c1.T, zk) - b1 <= 0,
            name=f"constraint_ineq1_{k+1}"
        )
        
    # Adds constraint of constraint (1/5)sx - 1^Tx <= 0
    model.addConstr(
        (1/5)*sx - gp.quicksum(x[i] for i in range(m)) <= 0,
        name=f"constraint_ineq2"
    )
    # Adds constraint of constraint 1^Tx - sx <= 0
    model.addConstr(
        gp.quicksum(x[i] for i in range(m)) - sx <= 0,
        name=f"constraint_ineq3"
    )

    # Adds constraint of constraint x >= 0
    for i in range(m):
        model.addConstr(
            x[i] >= 0,
            name=f"constraint_ineq4_{i+1}"
        )

    # Adds binary cut if doing so in outer approximation
    for k in range(len(B)):
        yk = B[k]
        Bk = [i for i in range(n) if yk[i] == 1]
        Nk = [i for i in range(n) if yk[i] == 0]
        model.addConstr(
            gp.quicksum(y[i] for i in Bk) - gp.quicksum(y[i] for i in Nk) <= len(Bk) - 1,
            name=f"constraint_ineq5_{k+1}"
        )

    # Adds constraint of contraint 1^Ty = sy from problem P
    model.addConstr(gp.quicksum(y[i] for i in range(n)) - sy == 0, name=f"constraint_eq")

    # Configures log to not show messages on terminal
    model.setParam("OutputFlag", 0)

    # Defines SoftMemLimit (in GigaBytes)
    model.setParam("SoftMemLimit", 8)

    # Defines DualReductions to 0 to obtain a more definite conclusion when infeasible or unbounded
    model.setParam("DualReductions", 0)

    # Optimizes model
    model.optimize()

    # Checks if model was optimized successfully
    if model.status == GRB.OPTIMAL:
        # Extracts model variables
        alpha_value = alpha.X 
        x_values = np.array([x[i].X for i in range(m)])
        y_values = np.array([y[i].X for i in range(n)])
        return [alpha_value, x_values, y_values]
    else:
        raise RuntimeError(f"Model was not optimized successfully. Status: {model.status}")

In [14]:
def extended_cutting_plane(m, n, problem_data, L0, epsilon, verbose=True):
    zl = -np.inf
    zu = np.inf
    z0 = random_viable_point(m, n, problem_data)
    L = [elem for elem in L0]
    L.append(z0)
    z_star = None
    k = 1
    while (zu - zl > epsilon):
        if verbose:
            print("-------------------------")
            print(f"Iteration k = {k}")
        if insufficient_memory():
            print("Insufficient memory to continue the function.")
            return z_star
        alpha, x, y = master_problem(m=m, n=n, problem_data=problem_data, L=L)
        zl = alpha
        if is_xy_viable(x, y, m, n, problem_data) and fxy(x, y, problem_data["Q"]) < zu:
            z_star = [x[i] for i in range(m)] + [y[i] for i in range(n)]
            zu = fxy(x, y, problem_data["Q"])
        k += 1
        if verbose:
            print(f"zl: {zl}")
            print(f"zu: {zu}")
            print(f"zu - zl: {zu - zl}")
            print("-------------------------")
        z = [x[i] for i in range(m)] + [y[i] for i in range(n)]
        L.append(z)
    if verbose:
        print("Solution found.")
    return z_star, k

In [15]:
def pyk_problem_solver(y, m, n, problem_data):
    # Extracts problem data
    Q = problem_data["Q"]
    A = problem_data["A"]
    c1 = problem_data["c1"]
    sy = problem_data["sy"]
    sx = problem_data["sx"]
    b1 = problem_data["b1"]

    # Creates a new model
    model = gp.Model("minlp")

    # Creates variables
    x = model.addVars(m, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="x")
    z = [x[i] for i in range(m)] + [y[i] for i in range(n)]
    z_arr = np.array(z)

    # Defines objective function
    objective = np.dot(z_arr.T, np.dot(Q, z_arr))

    # Sets objective
    model.setObjective(objective, GRB.MINIMIZE)

    # Adds constraint of constraint z^TAz - c^Tz <= b1 from problem P
    model.addConstr(
        np.dot(z_arr.T, np.dot(A, z_arr)) - np.dot(c1.T, z_arr) - b1 <= 0,
        name=f"constraint_ineq1"
    )

    # Adds constraint of constraint (1/5)sx - 1^Tx <= 0
    model.addConstr(
        (1/5)*sx - gp.quicksum(x[i] for i in range(m)) <= 0,
        name=f"constraint_ineq2"
    )
    # Adds constraint of constraint 1^Tx - sx <= 0
    model.addConstr(
        gp.quicksum(x[i] for i in range(m)) - sx <= 0,
        name=f"constraint_ineq3"
    )

    # Adds constraint of constraint x >= 0
    for i in range(m):
        model.addConstr(
            x[i] >= 0,
            name=f"constraint_ineq4_{i+1}"
        )

    # Adds constraint of contraint 1^Ty = sy from problem P
    model.addConstr(gp.quicksum(y[i] for i in range(n)) - sy == 0, name=f"constraint_eq")

    # Configures log to not show messages on terminal
    model.setParam("OutputFlag", 0)

    # Defines SoftMemLimit (in GigaBytes)
    model.setParam('SoftMemLimit', 8)

    # Defines DualReductions to 0 to obtain a more definite conclusion when infeasible or unbounded
    model.setParam("DualReductions", 0)

    # Optimizes model
    model.optimize()

    # Checks if model was optimized successfully
    if model.status == GRB.OPTIMAL:
        # Extracts model variables
        x_values = np.array([x[i].X for i in range(m)])
        return x_values
    elif model.status == GRB.INFEASIBLE:
        return -1
    else:
        raise RuntimeError(f"Model was not optimized successfully. Status: {model.status}")

In [16]:
def pykf_problem_solver(y, m, n, problem_data):
    # Extracts problem data
    Q = problem_data["Q"]
    A = problem_data["A"]
    c1 = problem_data["c1"]
    sy = problem_data["sy"]
    sx = problem_data["sx"]
    b1 = problem_data["b1"]

    # Creates a new model
    model = gp.Model("minlp")

    # Creates variables
    u = model.addVar(lb=0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="u")
    x = model.addVars(m, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="x")
    z = [x[i] for i in range(m)] + [y[i] for i in range(n)]
    z_arr = np.array(z)

    # Defines objective function
    objective = u

    # Sets objective
    model.setObjective(objective, GRB.MINIMIZE)

    # Adds constraint of constraint z^TAz - c^Tz - b1 <= u from problem P_yk^F
    model.addConstr(
        np.dot(z_arr.T, np.dot(A, z_arr)) - np.dot(c1.T, z_arr) - b1 <= u,
        name=f"constraint_ineq1"
    )

    # Adds constraint of constraint (1/5)sx - 1^Tx <= 0
    model.addConstr(
        (1/5)*sx - gp.quicksum(x[i] for i in range(m)) <= 0,
        name=f"constraint_ineq2"
    )
    # Adds constraint of constraint 1^Tx - sx <= 0
    model.addConstr(
        gp.quicksum(x[i] for i in range(m)) - sx <= 0,
        name=f"constraint_ineq3"
    )

    # Adds constraint of constraint x >= 0
    for i in range(m):
        model.addConstr(
            x[i] >= 0,
            name=f"constraint_ineq4_{i+1}"
        )
    
    # Adds constraint of constraint u >= 0
    model.addConstr(
        u >= 0,
        name=f"constraint_ineq5"
    )

    # Adds constraint of contraint 1^Ty = sy from problem P
    model.addConstr(gp.quicksum(y[i] for i in range(n)) - sy == 0, name=f"constraint_eq")

    # Configures log to not show messages on terminal
    model.setParam("OutputFlag", 0)

    # Defines SoftMemLimit (in GigaBytes)
    model.setParam('SoftMemLimit', 8)

    # Defines DualReductions to 0 to obtain a more definite conclusion when infeasible or unbounded
    model.setParam("DualReductions", 0)

    # Optimizes model
    model.optimize()

    # Checks if model was optimized successfully
    if model.status == GRB.OPTIMAL:
        # Extracts model variables
        x_values = np.array([x[i].X for i in range(m)])
        return x_values
    elif model.status == GRB.INFEASIBLE:
        return -1
    else:
        raise RuntimeError(f"Model was not optimized successfully. Status: {model.status}")

In [17]:
def outer_approximation(m, n, problem_data, L0, epsilon, verbose=True):
    zl = -np.inf
    zu = np.inf
    z0 = random_viable_point(m, n, problem_data)
    L = [elem for elem in L0]
    L.append(z0)
    z_star = None
    k = 1
    while (zu - zl > epsilon):
        if verbose:
            print("-------------------------")
            print(f"Iteration k = {k}")
        if insufficient_memory():
            print("Insufficient memory to continue the function.")
            return z_star
        alpha, x, y = master_problem(m=m, n=n, problem_data=problem_data, L=L)
        zl = alpha
        x_bar = pyk_problem_solver(y=y, m=m, n=n, problem_data=problem_data)
        if type(x_bar) != int:
            if fxy(x_bar, y, problem_data["Q"]) < zu:
                z_star = [x_bar[i] for i in range(m)] + [y[i] for i in range(n)]
                zu = fxy(x_bar, y, problem_data["Q"])
        else:
            x_bar = pykf_problem_solver(y=y, m=m, n=n, problem_data=problem_data)
        k += 1
        if verbose:
            print(f"zl: {zl}")
            print(f"zu: {zu}")
            print(f"zu - zl: {zu - zl}")
            print("-------------------------")
        z = [x_bar[i] for i in range(m)] + [y[i] for i in range(n)]
        L.append(z)
    if verbose:
        print("Solution found.")
    return z_star, k

In [18]:
def outer_approximation_binary(m, n, problem_data, L0, epsilon, verbose=True):
    zl = -np.inf
    zu = np.inf
    z0 = random_viable_point(m, n, problem_data)
    L = [elem for elem in L0]
    L.append(z0)
    B = []
    z_star = None
    k = 1
    while (zu - zl > epsilon):
        if verbose:
            print("-------------------------")
            print(f"Iteration k = {k}")
        if insufficient_memory():
            print("Insufficient memory to continue the function.")
            return z_star
        alpha, x, y = master_problem(m=m, n=n, problem_data=problem_data, L=L, B=B)
        zl = alpha
        x_bar = pyk_problem_solver(y=y, m=m, n=n, problem_data=problem_data)
        if type(x_bar) != int:
            if fxy(x_bar, y, problem_data["Q"]) < zu:
                z_star = [x_bar[i] for i in range(m)] + [y[i] for i in range(n)]
                zu = fxy(x_bar, y, problem_data["Q"])
        else:
            B.append(y)
        k += 1
        if verbose:
            print(f"zl: {zl}")
            print(f"zu: {zu}")
            print(f"zu - zl: {zu - zl}")
            print("-------------------------")
        z = [x_bar[i] for i in range(m)] + [y[i] for i in range(n)]
        L.append(z)
    if verbose:
        print("Solution found.")
    return z_star, k

In [19]:
def benders_master_problem(m, n, problem_data, KV, KI):
    # Extracts problem data
    Q = problem_data["Q"]
    A = problem_data["A"]
    c1 = problem_data["c1"]
    sy = problem_data["sy"]
    sx = problem_data["sx"]
    b1 = problem_data["b1"]

    # Partitioning matrix Q
    Q1 = Q[:m, :m]
    Q2 = Q[:m, m:]
    Q3 = Q[m:, :m]
    Q4 = Q[m:, m:]

    # Partitioning matrix A
    A1 = A[:m, :m]
    A2 = A[:m, m:]
    A3 = A[m:, :m]
    A4 = A[m:, m:]

    # Partitioning vector c1
    c1x = c1[:m]
    c1y = c1[m:]

    # Creates a new model
    model = gp.Model("milp")

    # Creates variables
    alpha = model.addVar(lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="alpha")
    y = model.addVars(n, lb=0, ub=1, vtype=GRB.BINARY, name="y")
    y_arr = np.array([y[i] for i in range(n)])

    # Defines objective function
    objective = alpha

    # Sets objective
    model.setObjective(objective, GRB.MINIMIZE)

    for k in range(len(KV)):
        xk, yk, lambdak = KV[k]
        zk = np.array([xk[i] for i in range(m)] + [yk[i] for i in range(n)])
        grad_fy = np.dot(Q2.T, xk) + np.dot(Q3, xk) + np.dot(Q4, yk) + np.dot(Q4.T, yk)
        grad_gy = np.dot(A2.T, xk) + np.dot(A3, xk) + np.dot(A4, yk) + np.dot(A4.T, yk) - c1y
        model.addConstr(
            np.dot(grad_fy.T, y_arr - yk) + np.dot(zk.T, np.dot(Q, zk))
            + lambdak * (np.dot(grad_gy.T, y_arr - yk) + np.dot(zk.T, np.dot(A, zk)) - np.dot(c1.T, zk) - b1) <= alpha,
            name=f"constraint_ineq1_KV_{k+1}"
        )
    
    for k in range(len(KI)):
        xk, yk, muk = KI[k]
        zk = np.array([xk[i] for i in range(m)] + [yk[i] for i in range(n)])
        grad_fy = np.dot(Q2.T, xk) + np.dot(Q3, xk) + np.dot(Q4, yk) + np.dot(Q4.T, yk)
        grad_gy = np.dot(A2.T, xk) + np.dot(A3, xk) + np.dot(A4, yk) + np.dot(A4.T, yk) - c1y
        model.addConstr(
            muk * (np.dot(grad_gy.T, y_arr - yk) + np.dot(zk.T, np.dot(A, zk)) - np.dot(c1.T, zk) - b1) <= 0,
            name=f"constraint_ineq1_KV_{k+1}"
        )

    # Adds constraint of contraint 1^Ty = sy from problem P
    model.addConstr(gp.quicksum(y[i] for i in range(n)) - sy == 0, name=f"constraint_eq")

    # Configures log to not show messages on terminal
    model.setParam("OutputFlag", 0)

    # Defines SoftMemLimit (in GigaBytes)
    model.setParam("SoftMemLimit", 8)

    # Defines DualReductions to 0 to obtain a more definite conclusion when infeasible or unbounded
    model.setParam("DualReductions", 0)

    # Optimizes model
    model.optimize()

    # Checks if model was optimized successfully
    if model.status == GRB.OPTIMAL:
        # Extracts model variables
        alpha_value = alpha.X 
        y_values = np.array([y[i].X for i in range(n)])
        return [alpha_value, y_values]
    else:
        raise RuntimeError(f"Model was not optimized successfully. Status: {model.status}")

In [20]:
def benders_pyk_problem_solver(y, m, n, problem_data):
    # Extracts problem data
    Q = problem_data["Q"]
    A = problem_data["A"]
    c1 = problem_data["c1"]
    sy = problem_data["sy"]
    sx = problem_data["sx"]
    b1 = problem_data["b1"]

    # Creates a new model
    model = gp.Model("minlp")

    # Creates variables
    x = model.addVars(m, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="x")
    z = [x[i] for i in range(m)] + [y[i] for i in range(n)]
    z_arr = np.array(z)

    # Defines objective function
    objective = np.dot(z_arr.T, np.dot(Q, z_arr))

    # Sets objective
    model.setObjective(objective, GRB.MINIMIZE)

    # Adds constraint of constraint z^TAz - c^Tz <= b1 from problem P
    model.addConstr(
        np.dot(z_arr.T, np.dot(A, z_arr)) - np.dot(c1.T, z_arr) - b1 <= 0,
        name=f"constraint_ineq1"
    )

    # Adds constraint of constraint (1/5)sx - 1^Tx <= 0
    model.addConstr(
        (1/5)*sx - gp.quicksum(x[i] for i in range(m)) <= 0,
        name=f"constraint_ineq2"
    )
    # Adds constraint of constraint 1^Tx - sx <= 0
    model.addConstr(
        gp.quicksum(x[i] for i in range(m)) - sx <= 0,
        name=f"constraint_ineq3"
    )

    # Adds constraint of constraint x >= 0
    for i in range(m):
        model.addConstr(
            x[i] >= 0,
            name=f"constraint_ineq4_{i+1}"
        )

    # Adds constraint of contraint 1^Ty = sy from problem P
    model.addConstr(gp.quicksum(y[i] for i in range(n)) - sy == 0, name=f"constraint_eq")

    # Configures log to not show messages on terminal
    model.setParam("OutputFlag", 0)

    # Defines SoftMemLimit (in GigaBytes)
    model.setParam('SoftMemLimit', 8)

    # Defines DualReductions to 0 to obtain a more definite conclusion when infeasible or unbounded
    model.setParam("DualReductions", 0)

    # Makes Gurobi compute the dual multipliers for continuous convex QCPs by solving the corresponding KKT problem
    model.setParam("QCPDual", 1)

    # Optimizes model
    model.optimize()

    # Checks if model was optimized successfully
    if model.status == GRB.OPTIMAL:
        # Extracts model variables
        x_values = np.array([x[i].X for i in range(m)])
        lambda_values = model.getQConstrs()[0].QCPi
        return [x_values, lambda_values]
    elif model.status == GRB.INFEASIBLE:
        return -1
    else:
        raise RuntimeError(f"Model was not optimized successfully. Status: {model.status}")

In [21]:
def benders_pykf_problem_solver(y, m, n, problem_data):
    # Extracts problem data
    Q = problem_data["Q"]
    A = problem_data["A"]
    c1 = problem_data["c1"]
    sy = problem_data["sy"]
    sx = problem_data["sx"]
    b1 = problem_data["b1"]

    # Creates a new model
    model = gp.Model("minlp")

    # Creates variables
    u = model.addVar(lb=0, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="u")
    x = model.addVars(m, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="x")
    z = [x[i] for i in range(m)] + [y[i] for i in range(n)]
    z_arr = np.array(z)

    # Defines objective function
    objective = u

    # Sets objective
    model.setObjective(objective, GRB.MINIMIZE)

    # Adds constraint of constraint z^Az - c^Tz - b1 <= u from problem P_yk^F
    model.addConstr(
        np.dot(z_arr.T, np.dot(A, z_arr)) - np.dot(c1.T, z_arr) - b1 <= u,
        name=f"constraint_ineq1"
    )

    # Adds constraint of constraint (1/5)sx - 1^Tx <= 0
    model.addConstr(
        (1/5)*sx - gp.quicksum(x[i] for i in range(m)) <= 0,
        name=f"constraint_ineq2"
    )
    # Adds constraint of constraint 1^Tx - sx <= 0
    model.addConstr(
        gp.quicksum(x[i] for i in range(m)) - sx <= 0,
        name=f"constraint_ineq3"
    )

    # Adds constraint of constraint x >= 0
    for i in range(m):
        model.addConstr(
            x[i] >= 0,
            name=f"constraint_ineq4_{i+1}"
        )
    
    # Adds constraint of constraint u >= 0
    model.addConstr(
        u >= 0,
        name=f"constraint_ineq5"
    )

    # Adds constraint of contraint 1^Ty = sy from problem P
    model.addConstr(gp.quicksum(y[i] for i in range(n)) - sy == 0, name=f"constraint_eq")

    # Configures log to not show messages on terminal
    model.setParam("OutputFlag", 0)

    # Defines SoftMemLimit (in GigaBytes)
    model.setParam('SoftMemLimit', 8)

    # Defines DualReductions to 0 to obtain a more definite conclusion when infeasible or unbounded
    model.setParam("DualReductions", 0)

    # Makes Gurobi compute the dual multipliers for continuous convex QCPs by solving the corresponding KKT problem
    model.setParam("QCPDual", 1)

    # Optimizes model
    model.optimize()

    # Checks if model was optimized successfully
    if model.status == GRB.OPTIMAL:
        # Extracts model variables
        x_values = np.array([x[i].X for i in range(m)])
        mu_values = model.getQConstrs()[0].QCPi
        return [x_values, mu_values]
    elif model.status == GRB.INFEASIBLE:
        return -1
    else:
        raise RuntimeError(f"Model was not optimized successfully. Status: {model.status}")

In [22]:
def generalized_benders_decomposition(m, n, problem_data, epsilon, verbose=True):
    zl = -np.inf
    zu = np.inf
    KV = []
    KI = []
    z0 = random_viable_point(m, n, problem_data)
    y0 = z0[m:]
    solution = benders_pyk_problem_solver(y=y0, m=m, n=n, problem_data=problem_data)
    if type(solution) == int and solution == -1:
        x0, mu0 = benders_pykf_problem_solver(y=y0, m=m, n=n, problem_data=problem_data)
        KI.append([x0, y0, mu0])
    else:
        x0, lambda0 = solution[0], solution[1]
        KV.append([x0, y0, lambda0])
    z_star = None
    k = 1
    while (zu - zl > epsilon):
        if verbose:
            print("-------------------------")
            print(f"Iteration k = {k}")
        if insufficient_memory():
            print("Insufficient memory to continue the function.")
            return z_star
        alphak, yk = benders_master_problem(m=m, n=n, problem_data=problem_data, KV=KV, KI=KI)
        zl = alphak
        solution = benders_pyk_problem_solver(y=yk, m=m, n=n, problem_data=problem_data)
        if type(solution) != int:
            xk = solution[0]
            lambdak = solution[1]
            if fxy(xk, yk, problem_data["Q"]) < zu:
                z_star = [xk[i] for i in range(m)] + [yk[i] for i in range(n)]
                zu = fxy(xk, yk, problem_data["Q"])
            KV.append([xk, yk, lambdak])
        else:
            xk, muk = benders_pykf_problem_solver(y=yk, m=m, n=n, problem_data=problem_data)
            KI.append([xk, yk, muk])
        k += 1
        if verbose:
            print(f"zl: {zl}")
            print(f"zu: {zu}")
            print(f"zu - zl: {zu - zl}")
            print("-------------------------")
    if verbose:
        print("Solution found.")
    return z_star, k

In [23]:
def gurobi_solver(m, n, problem_data):
    # Extracts problem data
    Q = problem_data["Q"]
    A = problem_data["A"]
    c1 = problem_data["c1"]
    sy = problem_data["sy"]
    sx = problem_data["sx"]
    b1 = problem_data["b1"]
    
    # Creates a new model
    model = gp.Model("minlp")

    # Creates variables
    x = model.addVars(m, lb=-GRB.INFINITY, ub=GRB.INFINITY, vtype=GRB.CONTINUOUS, name="x")
    y = model.addVars(n, lb=0, ub=1, vtype=GRB.BINARY, name="y")
    z = [x[i] for i in range(m)] + [y[i] for i in range(n)]
    z_arr = np.array(z)

    # Defines objective function
    # objective = gp.quicksum(Q[i, j] * z[i] * z[j] for i in range(n) for j in range(n))
    objective = np.dot(z_arr.T, np.dot(Q, z_arr))

    # Sets objective
    model.setObjective(objective, GRB.MINIMIZE)

    # Adds constraint z^TAz - c^Tz <= b1
    model.addConstr(
        np.dot(z_arr.T, np.dot(A, z_arr)) - np.dot(c1, z_arr) - b1 <= 0,
        name="constraint_ineq1"
    )

    # Adds constraint sx/5 <= 1^Tx
    model.addConstr(
        (1/5)*sx - gp.quicksum(x[i] for i in range(m)) <= 0,
        name="constraint_ineq2"
    )

    # Adds constraint 1^Tx <= sx
    model.addConstr(
        gp.quicksum(x[i] for i in range(m)) - sx <= 0,
        name="constraint_ineq3"
    )

    for i in range(m):
        model.addConstr(
            x[i] >= 0,
            name=f"constraint_ineq4_{i+1}"
        )

    # Adds constraint of contraint 1^Ty = sy
    model.addConstr(gp.quicksum(y[i] for i in range(n)) - sy == 0, name=f"constraint_eq")

    # Configures log to not show messages on terminal
    model.setParam("OutputFlag", 0)

    # Defines SoftMemLimit (in GigaBytes)
    model.setParam('SoftMemLimit', 8)

    # Optimizes model
    model.optimize()

    # Checks if model was optimized successfully
    if model.status == GRB.OPTIMAL:
        # Extracts model variables
        x_values = np.array([x[i].X for i in range(m)])
        y_values = np.array([y[i].X for i in range(n)])
        return [x_values, y_values]
    else:
        raise RuntimeError(f"Model was not optimized successfully. Status: {model.status}")

## Testing

### Setting Parameters

In [24]:
# m = 10
# n = 8

# m = 15
# n = 12

m = 20
n = 16

problem_data = create_problem_instance(m=m, n=n)

L = [random_point(m, n)]

epsilon = 10 ** -2

### Executing Algorithms

In [ ]:
z_ecp_star, k_ecp = extended_cutting_plane(m, n, problem_data, L, epsilon, verbose=True)

In [ ]:
z_op_star, k_op = outer_approximation(m, n, problem_data, L, epsilon, verbose=False)

In [ ]:
z_opb_star, k_opb = outer_approximation_binary(m, n, problem_data, L, epsilon, verbose=False)

In [ ]:
z_gbd_star, k_gbd = generalized_benders_decomposition(m, n, problem_data, epsilon, verbose=False)

In [ ]:
x_g_star, y_g_star = gurobi_solver(m, n, problem_data)

### Checking Solution Viability and Objective Value

In [ ]:
# Extended Cutting Plane Solution
print(is_xy_viable(z_ecp_star[:m], z_ecp_star[m:], m, n, problem_data))
print(fz(z_ecp_star, problem_data["Q"]))

# Outer Approximation Solution
print(is_xy_viable(z_op_star[:m], z_op_star[m:], m, n, problem_data))
print(fz(z_op_star, problem_data["Q"]))

# Outer Approximation with Binary Cut Solution
print(is_xy_viable(z_opb_star[:m], z_opb_star[m:], m, n, problem_data))
print(fz(z_opb_star, problem_data["Q"]))

# Generalized Benders Decomposition Solution
print(is_xy_viable(z_gbd_star[:m], z_gbd_star[m:], m, n, problem_data))
print(fz(z_gbd_star, problem_data["Q"]))

# Gurobi Solution
print(is_xy_viable(x_g_star, y_g_star, m, n, problem_data))
print(fxy(x_g_star, y_g_star, problem_data["Q"]))

## Producing Report Data

### Utility Functions

In [26]:
def process_data(algorithm_name, k, end_time, start_time, z_star):
    algorithm.append(algorithm_name)
    num_iteration.append(k)
    computational_time.append(end_time - start_time)
    objective_function_value.append(fxy(z_star[:m], z_star[m:], problem_data["Q"]))

In [27]:
def latex_solution(z, m, n):
    x = numpy_to_latex(z[:m])
    y = numpy_to_latex(z[m:])
    return x, y

In [28]:
def create_solution_txt_file(algorithm_name, z, m, n):
    z_arr = np.array(z)
    latex_x, latex_y = latex_solution(z_arr, m, n)
    with open(f"./Data/x_{algorithm_name}_m={m}_n={n}.txt", "w") as file:
        file.write(latex_x)
    with open(f"./Data/y_{algorithm_name}_m={m}_n={n}.txt", "w") as file:
        file.write(latex_y)

In [29]:
def create_vector_matrix_txt_file(data_name, data, m, n):
    with open(f"./Data/{data_name}_m={m}_n={n}.txt", "w") as file:
        file.write(data)

### Producing Data

In [30]:
problem_instances = [[10, 8], [15, 12], [20, 16]]

epsilon = 10 ** -2

In [31]:
algorithm = []
num_iteration = []
computational_time = []
objective_function_value = []

In [36]:
import time

m, n = problem_instances[2]
problem_data = create_problem_instance(m=m, n=n)
L = [random_point(m, n)]

ecp_start_time = time.time()
z_ecp_star, k_ecp = extended_cutting_plane(m, n, problem_data, L, epsilon, verbose=True)
ecp_end_time = time.time()
process_data("ECP", k_ecp, ecp_end_time, ecp_start_time, z_ecp_star)

op_start_time = time.time()
z_op_star, k_op = outer_approximation(m, n, problem_data, L, epsilon, verbose=True)
op_end_time = time.time()
process_data("OP", k_op, op_end_time, op_start_time, z_op_star)

opb_start_time = time.time()
z_opb_star, k_opb = outer_approximation_binary(m, n, problem_data, L, epsilon, verbose=True)
opb_end_time = time.time()
process_data("OPB", k_opb, opb_end_time, opb_start_time, z_opb_star)

gbd_start_time = time.time()
z_gbd_star, k_gbd = generalized_benders_decomposition(m, n, problem_data, epsilon, verbose=True)
gbd_end_time = time.time()
process_data("GBD", k_gbd, gbd_end_time, gbd_start_time, z_gbd_star)

g_start_time = time.time()
x_g_star, y_g_star = gurobi_solver(m, n, problem_data)
z_g_star = [x_g_star[i] for i in range(m)] + [y_g_star[i] for i in range(n)]
g_end_time = time.time()
process_data("Gurobi", "-", g_end_time, g_start_time, z_g_star)

Qeigenvalues, Qeigenvectors = np.linalg.eig(problem_data["Q"])
Aeigenvalues, Aeigenvectors = np.linalg.eig(problem_data["A"])

latex_MatrixQ = numpy_to_latex(problem_data["Q"])
create_vector_matrix_txt_file("matrixQ", latex_MatrixQ, m, n)

latex_Qeigenvalues = numpy_to_latex(Qeigenvalues)
create_vector_matrix_txt_file("Qeigenvalues", latex_Qeigenvalues, m, n)

latex_MatrixA = numpy_to_latex(problem_data["A"])
create_vector_matrix_txt_file("matrixA", latex_MatrixA, m, n)

latex_Aeigenvalues = numpy_to_latex(Aeigenvalues)
create_vector_matrix_txt_file("Aeigenvalues", latex_Aeigenvalues, m, n)

latex_c1 = numpy_to_latex(problem_data["c1"])
create_vector_matrix_txt_file("c1", latex_c1, m, n)

create_solution_txt_file("ECP", z_ecp_star, m, n)
create_solution_txt_file("OP", z_op_star, m, n)
create_solution_txt_file("OPB", z_opb_star, m, n)
create_solution_txt_file("GBD", z_gbd_star, m, n)
create_solution_txt_file("Gurobi", z_g_star, m, n)

-------------------------
Iteration k = 1
zl: -519.4397759092383
zu: inf
zu - zl: inf
-------------------------
-------------------------
Iteration k = 2
zl: -202.02689828797853
zu: inf
zu - zl: inf
-------------------------
-------------------------
Iteration k = 3
zl: -41.20833394781033
zu: inf
zu - zl: inf
-------------------------
-------------------------
Iteration k = 4
zl: 33.96073247012318
zu: inf
zu - zl: inf
-------------------------
-------------------------
Iteration k = 5
zl: 60.33935061806647
zu: inf
zu - zl: inf
-------------------------
-------------------------
Iteration k = 6
zl: 65.05764150162756
zu: 102.9739521819931
zu - zl: 37.91631068036554
-------------------------
-------------------------
Iteration k = 7
zl: 66.43378295464574
zu: 102.9739521819931
zu - zl: 36.54016922734736
-------------------------
-------------------------
Iteration k = 8
zl: 67.74620705808422
zu: 102.9739521819931
zu - zl: 35.227745123908875
-------------------------
-----------------------

### Making CSV

In [37]:
import pandas as pd

data = {
    "algorithm": algorithm,
    "num_iteration": num_iteration,
    "computational_time": computational_time,
    "objective_function_value": objective_function_value
}

df = pd.DataFrame(data)

df.to_csv('data.csv', index=False)